In [59]:
import numpy as np
import string
from itertools import permutations, combinations, product

def hypergraph_to_tensor(edges, num_vertices):
    m = len(edges[0])
    if not edges: return np.zeros((num_vertices,) * m, dtype=float)        
    shape = (num_vertices,) * m
    A = np.zeros(shape, dtype=float)
    for edge in edges:
        if any(v >= num_vertices for v in edge): raise ValueError("Edge index out of bounds.")
        for p in permutations(edge): A[p] = 1
    return A

def tensor_congruence_transform(A, Q):
    m, n_A = A.ndim, A.shape[0]
    n_Q = Q.nrows()
    if n_Q != n_A: raise ValueError(f"Matrix Q dim ({n_Q}) must match Tensor A dim ({n_A}).")
    Q_np = np.array(Q)
    j_indices, i_indices = string.ascii_lowercase[m:2*m], string.ascii_lowercase[0:m]
    einsum_str = f'{j_indices},' + ','.join([f'{i_indices[k]}{j_indices[k]}' for k in range(m)]) + f'->{i_indices}'
    return np.einsum(einsum_str, A, *([Q_np] * m))

def tensor_to_hypergraph_edges(A, tolerance=1e-6):
    m = A.ndim
    n = A.shape[0]
    edges = []
    for comb in combinations(range(n), m):
        if abs(A[comb]-1) < tolerance:
            edges.append(list(comb))
    return edges

def tensor_subs(n,A,sub_map):
    for (a,b,c) in combinations(range(n), 3):
        A[a,b,c] = A[a,b,c].subs(sub_map)
    return A

def tensor_print(n,A2):
    for (a,b,c) in combinations(range(n), 3):
        print(f"({a}, {b}, {c}) entry:")
        print(A2[a,b,c])

def R_gm(n):
    if n % 2 != 0:
        raise ValueError(f"n must be even, got {n}")
    if n < 4:
        raise ValueError(f"n must be >= 4, got {n}")        
    J_n = matrix(QQ, n, n, [1] * (n^2))
    I_n = identity_matrix(QQ, n)    
    return (2/n) * J_n - I_n

def R_wqh(p):
    if p < 1:
        raise ValueError(f"p must be >= 1, got {p}")    
    I_p = identity_matrix(QQ, p)
    J_p = matrix(QQ, p, p, [1] * (p^2))    
    A = I_p - (1/p) * J_p  
    B = (1/p) * J_p      
    return block_matrix(QQ, [[A, B], [B, A]])

def R_sg(n, m=0):    
    if n % 2 != 0:
        raise ValueError("n must be even (total matrix dimension).")
    b = n // 2 
    I2 = identity_matrix(QQ, 2)
    J2 = matrix(QQ, 2, 2, [1]*4)  
    O2 = matrix(QQ, 2, 2, 0)
    Y2 = 2*I2 - J2 
    first_row_blocks = [J2] + [O2]*(b-2) + [Y2]
    blocks = []
    for r in range(b):
        row_blocks = []
        for c in range(b):
            idx = (c - r) % b
            row_blocks.append(first_row_blocks[idx])
        blocks.append(row_blocks)
    Q_half = (QQ(1)/2) * block_matrix(blocks)
    if m > 0:
        I_1 = identity_matrix(QQ, 1)
        I_rest = identity_matrix(QQ, m - 1)
        Q_half = block_diagonal_matrix(I_1, Q_half, I_rest)
    return Q_half

def build_linear_system(Q, k, n):
    A1_unordered = np.zeros((n,)*k, dtype=object)    
    combos = list(combinations(range(1,n+1), k))
    names = [f"e_{'_'.join(map(str, combo))}" for combo in combos]    
    var(' '.join(names))    
    for combo in combos:
        sym_name = f"e_{'_'.join(map(str, combo))}"
        sym = globals().get(sym_name, SR(sym_name))
        for p in permutations(combo):
            A1_unordered[tuple(i - 1 for i in p)] = sym    
    A2 = tensor_congruence_transform(A1_unordered, Q_half)       
    print("------------------------")    
    coeff_rows_QQ = []
    for combo in combos:
        expr = SR(A2[tuple(i - 1 for i in combo)]).expand().simplify_full()
        row = [ QQ(expr.coefficient(SR(nm), 1)) for nm in names ]
        coeff_rows_QQ.append(row)    
    C_QQ = matrix(QQ, coeff_rows_QQ) 
    dens = [c.denominator() for row in coeff_rows_QQ for c in row]
    D = Integer(1) if not dens else lcm(dens)
    C_int = matrix(ZZ, [[ int((D * c).numerator()) for c in row ] for row in coeff_rows_QQ])
    return combos, names, C_int, int(D)
    
def MILP_search(Q_half,k,n):
    combos, names, C_int, D = build_linear_system(Q_half, k, n)
    mip = MixedIntegerLinearProgram(maximization=False)
    e = mip.new_variable(binary=True)  
    f = mip.new_variable(binary=True)    
    for t_idx in range(len(names)):
        row = C_int.row(t_idx)
        lin = sum(int(row[j]) * e[j] for j in range(len(names)))
        mip.add_constraint(lin == int(D) * f[t_idx])
    solutions = []
    incidence_structures = []
    iteration = 0    
    while True:
        try:
            mip.solve()
        except Exception as exc:
            print(f"\nSolver stopped after {iteration} iterations:", exc)
            break    
        e_sol = [int(mip.get_values(e[j])) for j in range(len(names))]
        f_sol = [int(mip.get_values(f[t])) for t in range(len(names))]    
        edges_original = [combos[i] for i, val in enumerate(e_sol) if val == 1]
        edges_transformed = [combos[i] for i, val in enumerate(f_sol) if val == 1]    
        IS_original = IncidenceStructure(edges_original)
        IS_transformed = IncidenceStructure(edges_transformed)    
        iteration += 1    
        is_iso = IS_original.is_isomorphic(IS_transformed)    
        is_new = True
        for (_, _, iso_pair) in incidence_structures:
            if IS_original.is_isomorphic(iso_pair[0]):
                is_new = False
                break    
        if is_new:
            incidence_structures.append((IS_original, IS_transformed, (IS_original, IS_transformed)))
            solutions.append((e_sol, f_sol))  
        mip.add_constraint(sum((1 - e[j]) if e_sol[j] else e[j] for j in range(len(names))) >= 1)    
        if iteration >= 20:
            print("\nStopping after limited runtime.")
            break    
    print(f"\nTotal distinct solutions found: {len(solutions)}")    
    unique_iso_types = []
    for i, (IS1, IS2, _) in enumerate(incidence_structures):
        if not any(IS1.is_isomorphic(prev[0]) for prev in unique_iso_types):
            unique_iso_types.append((IS1, IS2))            
            print("-" * 20)
            isom_trans = IS1.is_isomorphic(IS2)
            print("isomorphic pair:",isom_trans)
            print("num_vertices",len(IS1.ground_set()))
            print("num_edges",len(IS1.blocks()))
            print(IS1.blocks())
            print(IS2.blocks())

def exhaustive_search(Q_half,k,n):
    combos, names, C_int, D = build_linear_system(Q_half, k, n)    
    C_list = [[int(x) for x in row] for row in C_int.rows()]
    D_int = int(D)
    num_vars = len(names)
    num_constraints = len(C_list)    
    constraint_dependencies = [[] for _ in range(num_vars)]
    for r_idx, row in enumerate(C_list):
        last_var = -1
        for c_idx, val in enumerate(row):
            if val != 0:
                last_var = c_idx
        if last_var != -1:
            constraint_dependencies[last_var].append(r_idx)    
    solutions = []    
    def solve_recursively(var_idx, current_selection, current_row_sums):    
        if var_idx == num_vars:
            if sum(current_selection) >= 0:
                f_sol = [1 if s == D_int else 0 for s in current_row_sums]
                solutions.append((list(current_selection), f_sol))
            return    
        valid_0 = True
        for r_idx in constraint_dependencies[var_idx]:
            s = current_row_sums[r_idx]
            if s != 0 and s != D_int:
                valid_0 = False
                break        
        if valid_0:
            current_selection.append(0)
            solve_recursively(var_idx + 1, current_selection, current_row_sums)
            current_selection.pop()     
        new_sums = list(current_row_sums) 
        for r_idx in range(num_constraints):
            coeff = C_list[r_idx][var_idx]
            if coeff != 0:
                new_sums[r_idx] += coeff                
        valid_1 = True
        for r_idx in constraint_dependencies[var_idx]:
            s = new_sums[r_idx]
            if s != 0 and s != D_int:
                valid_1 = False
                break                
        if valid_1:
            current_selection.append(1)
            solve_recursively(var_idx + 1, current_selection, new_sums)
            current_selection.pop() # Backtrack    
    initial_sums = [0] * num_constraints
    solve_recursively(0, [], initial_sums)    
    print(f"Total solutions found: {len(solutions)}")    
    incidence_structures = []
    unique_iso_types = []    
    for e_sol, f_sol in solutions:
        edges_original = [combos[i] for i, val in enumerate(e_sol) if val == 1]
        edges_transformed = [combos[i] for i, val in enumerate(f_sol) if val == 1]
        # print(edges_original)
        IS_original = IncidenceStructure(edges_original)
        IS_transformed = IncidenceStructure(edges_transformed)        
        is_new = True
        for (existing_IS, _, _) in unique_iso_types:
            if IS_original.is_isomorphic(existing_IS):
                is_new = False
                break        
        if is_new:
            unique_iso_types.append((IS_original, IS_transformed, edges_original))  
            print(IS_original.blocks())
            print("-" * 20)    
    print(f"\nFinal Count: {len(unique_iso_types)} unique hypergraph isomorphism types.")

In [60]:
import time 

def resultant_sage(t_poly_comps,n):     
    macaulay2.eval('loadPackage "Resultants"')

    #Macaulay variables
    variable_names = ",".join([f"x{i}" for i in range(n)]) 
    macaulay2.eval(f'QQ[t][{variable_names}]')
 
    t_poly = "{" + ",".join([str(comp) for comp in t_poly_comps]) + "}"
    F = macaulay2(t_poly)
    restant = F.resultant()
    restant = restant.sage()
    return restant
    
def e_char_poly_generic(m, n):
    x_names = [f'x{i}' for i in range(1, n+1)]    
    indices = list(itertools.product(range(1, n+1), repeat=m))
    a_names = [f'a{"".join(map(str, idx))}' for idx in indices]    
    all_vars = x_names + a_names + ['t']
    R = PolynomialRing(QQ, names=all_vars)
    gens = R.gens()    
    x = gens[:n]
    a_vars = gens[n : n + n**m]
    t = gens[-1]
    a_map = {idx: var for idx, var in zip(indices, a_vars)}    
    equations = []    
    for i in range(1, n+1):
        poly_sum = 0
        rest_indices = list(itertools.product(range(1, n+1), repeat=m-1))
        for rest in rest_indices:
            full_idx = (i,) + rest
            coeff = a_map[full_idx]
            monomial = 1
            for var_idx in rest:
                monomial *= x[var_idx - 1] 
            poly_sum += coeff * monomial
        equations.append(poly_sum - t * x[i-1])
    equations.append(sum(xi**2 for xi in x) - 1)
    I = ideal(equations)
    f = I.elimination_ideal(list(x)).gens()[0]    
    if f != 0: 
        lc_t = lc_t = f.polynomial(t).leading_coefficient()
        f = f / lc_t 
    return f

def gen_elim_ideal(edges, n): 
    k = len(edges[0])    
    var_names = [f"x_{i}" for i in range(n+1)] + ["t"]
    R = PolynomialRing(QQ, var_names)    
    xs = R.gens()[:-1]
    t = R.gens()[-1]
    coord_vars = xs[1:n+1] 
    edge_poly = sum(prod(coord_vars[i] for i in ed) for ed in edges)    
    fermat = sum(var^2 for var in coord_vars)    
    if k % 2 == 0:
        t_poly_comps = []
        for i in range(n):
            deriv = edge_poly.derivative(coord_vars[i])
            poly = t * coord_vars[i] - deriv
            t_poly_comps.append(poly)
    else:
        t_poly_comps = []
        for i in range(n):
            deriv = edge_poly.derivative(coord_vars[i])
            poly = t * coord_vars[i] - deriv
            t_poly_comps.append(poly)
        
    t_poly_comps.append(fermat - 1)    
    I = ideal(t_poly_comps)
    elim_ideal = I.elimination_ideal([xs[i] for i in range(1, len(xs) )])    
    return elim_ideal.gens()[0]

def e_char_poly_res(edges, n, estimate_time=False):
    k = len(edges[0])
    
    if k % 2 == 0: #k even
        if k > 2:
            total_degree = ((k - 1) ** n - 1) / (k-2)
        else: 
            total_degree = n
        num_vars = n
        xs = [var(f"x{i}") for i in range(num_vars)]
        fermat_base = sum((xs[i])**2 for i in range(n))
        t_mults = [fermat_base**((k-2)//2) * xs[i] for i in range(n)]

    else: #k odd
        total_degree = 2 * ((k - 1) ** n - 1) / (k-2)
        num_vars = n + 1
        xs = [var(f"x{i}") for i in range(num_vars)]
        fermat_base = sum((xs[i])**2 for i in range(n))
        t_mults = [(xs[n])**(k-2) * xs[i] for i in range(n)]
        geom_constraint = fermat_base - (xs[n])**2
    edge_poly = sum(prod([xs[i] for i in ed]) for ed in edges)
    edge_diffs = [edge_poly.diff(xs[i]) for i in range(n)]
    
    macaulay2.eval('loadPackage "Resultants"')
    variable_names = ",".join([f"x{i}" for i in range(num_vars)])    
    points = []
    start_time = time.time()
    for t_val in range(total_degree + 1):
        
        t_poly_comps = []
        if k % 2 == 0:
            for i in range(n):
                eq = t_val * t_mults[i] - edge_diffs[i]
                t_poly_comps.append(eq)
                
        #Example: k = 4, edges = [(0,1,2,3)] (Single Edge of rank 4)
        # t_poly = "{ x1*x2*x3 - t * (x0^2+x1^2+x2^2+x3^2) * (x0), 
                        # x0*x2*x3 - t*(x0^2+x1^2+x2^2+x3^2) * (x1), 
                        # x0*x1*x3 - t * (x0^2+x1^2+x2^2+x3^2) * (x2), 
                        # x0*x1*x2 - t*(x0^2+x1^2+x2^2+x3^2) * (x3) }"    
        
        else:
            for i in range(n):
                eq = t_val * t_mults[i] - edge_diffs[i]
                t_poly_comps.append(eq)
            t_poly_comps.append(geom_constraint)

        #Example: k = 3, edges = [ (0,1,2) ] Single Edge of rank 3
        # t_poly = "{ x1*x2 - t * x3 * x0, x0*x2 - t* x3 * x1, x0*x1 - t * x3 * x2, ( x0^2 + x1^2 + x2^2 ) - x3^2 }"

        #Example: k = 3, edges = [ (0,1,2), (1,2,3) ]
        # t_poly = "{  x1*x2        - t * x4 * x0, 
                    # x0*x2 + x3*x2 - t* x4 * x1, 
                    # x0*x1 + x3*x1 - t* x4 * x2, 
                    # x1*x2         - t * x4 * x3, 
                    # ( x0^2 + x1^2 + x2^2 +x3^2 ) - x4^2 }"     
            
        macaulay2.eval(f'QQ[{variable_names}]')
        m2_list_str = "{" + ",".join([str(comp) for comp in t_poly_comps]) + "}"
        cmd = f"resultant({m2_list_str})"
        res_val = macaulay2(cmd)
        points.append((t_val, res_val.sage()))
        if estimate_time and t_val > 0:
            time_passed = time.time() - start_time
            avg_time = time_passed / t_val
            remaining_steps = total_degree - t_val
            time_remaining = avg_time * remaining_steps
            print(f"Computed t={t_val}/{total_degree}. Time Remaining: {datetime.timedelta(seconds=int(time_remaining))}", end='\r')
    R_t = PolynomialRing(QQ, 't')
    f = R_t.lagrange_polynomial(points)    
    if f != 0:
        return f / f.leading_coefficient()
    return f

In [61]:
# (E-)char-poly of a triangle of rank k=2
m = 2 
n = 3 

# First way: Via substitutions in the generic E-characteristic polynomial
poly = e_char_poly_generic(m,n)
print("Generic E-characteristic polynomial:")
print(poly)

print("-"*20)
print("(E-)characteristic polynomial:")
poly_1 = poly.subs(a13=1, a31=1, a12=1, a21=1, a23=1, a32=1, a11=0, a22=0, a33=0) 
print(poly_1)
print(poly_1.factor())

# Second way: Elimination of x1,x2,x3 in the system Ax=tx and x.x=1
# Does not agree with the characteristic polynomial which is det(tI-A). 
print("-"*20)
edges = [[0,1],[0,2],[1,2]]
poly_2 = gen_elim_ideal(edges,n)
print("Generator of the elimination ideal of x1,x2,x3 in <Ax-tx,x.x=1>:") 
print(poly_2)
print(poly_2.factor())

Generic E-characteristic polynomial:
t^3 + (-a11 - a22 - a33)*t^2 + (-a12*a21 + a11*a22 - a13*a31 - a23*a32 + a11*a33 + a22*a33)*t + a13*a22*a31 - a12*a23*a31 - a13*a21*a32 + a11*a23*a32 + a12*a21*a33 - a11*a22*a33
--------------------
(E-)characteristic polynomial:
t^3 - 3*t - 2
(t - 2) * (t + 1)^2
--------------------
Generator of the elimination ideal of x1,x2,x3 in <Ax-tx,x.x=1>:
t^2 - t - 2
(t - 2) * (t + 1)


In [62]:
# (E-)char-poly of a single edge of rank k=2
m = 2 
n = 2

# First way: Via substitutions in the generic E-characteristic polynomial
poly = e_char_poly_generic(m,n)
print("Generic E-characteristic polynomial:")
print(poly)

print("-"*20)
print("(E-)characteristic polynomial:")
poly_1 = poly.subs(a12=1, a21=1, a11=0, a22=0) 
print(poly_1)
print(poly_1.factor())

# Second way: Elimination of x1,x2,x3 in the system Ax=tx and x.x=1
# Does not agree with the characteristic polynomial which is det(tI-A). 
print("-"*20)
edges = [[0,1]]
poly_2 = gen_elim_ideal(edges,n)
print("Generator of the elimination ideal of x1,x2,x3 in < Ax-tx, x.x=1 >:") 
print(poly_2)
print(poly_2.factor())

Generic E-characteristic polynomial:
t^2 + (-a11 - a22)*t - a12*a21 + a11*a22
--------------------
(E-)characteristic polynomial:
t^2 - 1
(t - 1) * (t + 1)
--------------------
Generator of the elimination ideal of x1,x2,x3 in < Ax-tx, x.x=1 >:
t^2 - 1
(t - 1) * (t + 1)


In [63]:
#Example
# p. 1371 in Eigenvalues and invariants of tensors

# First way
n = 3
t = var("t")
xs = [var(f"x{i}") for i in range(n)] 
t_poly_comps = [ 
                x1^2 - t * x0 * x1, 
                x2^2 - t * x0 * x2, 
                x1^2 + x2^2 - x0^2
                ]
e_poly_1 = resultant_sage(t_poly_comps,n)
print(e_poly_1.factor())

# Second way
R.<x1, x2, t> = PolynomialRing(QQ)
I = ideal( 
        x1^2 - t*x1,
        x2^2 - t*x2,
        1 - x1^2 - x2^2
        )
e_poly_2 = I.elimination_ideal([x1, x2]).gens()[0]
print(e_poly_2.factor())

(-2) * (t - 1)^2 * (t + 1)^2 * (t^2 - 1/2)
(t - 1) * (t + 1) * (2*t^2 - 1)


In [64]:
# m = 3, n = 3
# E-char-poly of single edge 
# deg = 3 + 4 = 7 = 2^3 - 1 attains maximum

# First way: Elimination ideal
n = 3
edges = [ (0, 1, 2) ]
e_poly_1 = gen_elim_ideal(edges,n)
print(e_poly_1.factor())

# Second way: Resultant 
e_poly_2 = e_char_poly_res(edges,n)
print(e_poly_2.factor())

t * (3*t^2 - 1)
t^6 * (t^2 - 1/3)^4


In [65]:
# m = 3, n = 3
# Char-poly of single edge 

# First way: Elimination ideal
R.<x0,x1, x2, t> = PolynomialRing(QQ)
I = ideal(
        x1*x2 - t*x0^2,
        x0*x2 - t*x1^2,
        x0*x1 - t*x2^2,
        x0^2 + x1^2 + x2^2 - 1
        )
char_poly = I.elimination_ideal([x0, x1, x2]).gens()[0]
print(char_poly.factor())

# Second way: Resultant (already homogenous)
n = 3
t = var("t")
xs = [var(f"x{i}") for i in range(n)] 
t_poly_comps = [ 
        x1*x2 - t*x0^2,
        x0*x2 - t*x1^2,
        x0*x1 - t*x2^2
        ]
e_poly_1 = resultant_sage(t_poly_comps,n)
print(e_poly_1.factor())

(t - 1) * t * (t^2 + t + 1)
(t - 1)^3 * t^3 * (t^2 + t + 1)^3


In [66]:
# n = 4, m = 3 
# E-char-poly of Complete hypergraph of rank 3 and dimension 4

# First way: Elimination 
n = 4
edges = [ (0, 1, 2), (1, 2, 3), (0, 1, 3), (0, 2, 3) ]
e_poly_1 = gen_elim_ideal(edges,n)
print(e_poly_1.factor())

# Second way: Resultant (regular tensor)
e_poly_2 = e_char_poly_res(edges,n)
print(e_poly_2.factor())

t * (2*t - 3) * (2*t + 3) * (6*t^2 - 1) * (21*t^2 - 16)
(t - 3/2) * (t + 3/2) * t^8 * (t^2 - 16/21)^4 * (t^2 - 1/6)^6


In [67]:
# n = 4, m = 4
# E-char-poly of Single Edge of rank 4

# First way: Elimination 
n = 4
edges = edges = [ (0, 1, 2, 3) ] 
e_poly_1 = gen_elim_ideal(edges,n)
print(e_poly_1.factor())

# Second way: Resultant (irregular tensor)
e_poly_2 = e_char_poly_res(edges,n)
print(e_poly_2)

t * (4*t - 1) * (4*t + 1)
0


In [68]:
# m = 4, n = 5
n = 5
edges_1 = [ [0,1,2,3],[1,2,3,4] ]
e_poly_1 = gen_elim_ideal(edges_1,n)
print(e_poly_1.factor())

t * (8*t^2 - 1)


In [69]:
# m = 3, n = 4
n = 4
edges_2 = [ [0,1,2],[1,2,3] ]
e_poly_2 = gen_elim_ideal(edges_2,n)
print(e_poly_2.factor())

t * (3*t^2 - 2)


In [70]:
#Fano switching
k=2
Q = matrix(QQ, [
               [-1, 1, 1, 0, 1, 0, 0], 
               [0, -1, 1, 1, 0, 1, 0], 
               [0, 0, -1, 1, 1, 0, 1],
               [1, 0, 0, -1, 1, 1, 0], 
               [0, 1, 0, 0, -1, 1, 1], 
               [1, 0, 1, 0, 0, -1, 1],
               [1, 1, 0, 1, 0, 0, -1]
               ])

Q_half = (1/2) * Q
Q_half = Q_half.T
n = len(Q_half[0])

# MILP_search(Q_half,k,n)
# exhaustive_search(Q_half,k,n) 

#Fano Switching Small Test
I_1 = identity_matrix(QQ, 1)
Q_half = block_diagonal_matrix(I_1, Q_half)

I_m = identity_matrix(QQ, 2)
Q_half = block_diagonal_matrix(Q_half, I_m)

n = 10
ed_1 = [ [1, 2, 4], [2, 3, 5], [3, 4, 6], [4, 5, 7], [5, 6, 1], [6, 7, 2], [7, 1, 3], [1, 0, 8], [2, 0, 8], [4, 0, 8], [0, 8, 9] ]
A_1 = hypergraph_to_tensor(ed_1, n)
A_2 = tensor_congruence_transform(A_1,Q_half)
ed_2 = tensor_to_hypergraph_edges(A_2)
print(ed_2)

[[0, 3, 8], [0, 5, 8], [0, 6, 8], [0, 8, 9], [1, 2, 6], [1, 3, 4], [1, 5, 7], [2, 3, 7], [2, 4, 5], [3, 5, 6], [4, 6, 7]]


In [71]:
k = 1
I2 = matrix(QQ, [[1,0],[0,1]])
J2 = matrix(QQ, [[1,1],[1,1]])
Z2 = J2 - I2
blocks = [
         [-I2,  I2,  I2,  I2],
         [ I2, -Z2,  I2,  Z2],
         [ I2,  Z2, -Z2,  I2],
         [ I2,  I2,  Z2, -Z2]
         ]

Q_half = (QQ(1)/2) * block_matrix(blocks)
Q_half = Q_half.T
n = len(Q_half[0])
# MILP_search(Q_half,k,n)
# exhaustive_search(Q_half,k,n)   

# Cube switching Small Test
I_1 = identity_matrix(QQ, 1)
Q_half = block_diagonal_matrix(I_1, Q_half)
I_m = identity_matrix(QQ, 2)
Q_half = block_diagonal_matrix(Q_half, I_m)

n = len(Q_half[0])
ed_1 =  [ [2, 0, 9, 10], [3, 0, 9, 10], [6, 0, 9, 10], [7, 0, 9, 10], 
          [1, 7, 9, 10], [2, 6, 9, 10], [2, 8, 9, 10], [3, 5, 9, 10],
          [4, 5, 9, 10], [4, 6, 9, 10], [4, 7, 9, 10], [4, 8, 9, 10],
          [5, 6, 9, 10], [6, 7, 9, 10]
        ]
#
A_1 = hypergraph_to_tensor(ed_1, n)
A_2 = tensor_congruence_transform(A_1,Q_half)
ed_2 = tensor_to_hypergraph_edges(A_2)
print(ed_2)

[[0, 1, 9, 10], [0, 3, 9, 10], [0, 6, 9, 10], [0, 8, 9, 10], [1, 2, 9, 10], [1, 7, 9, 10], [1, 8, 9, 10], [2, 4, 9, 10], [2, 6, 9, 10], [2, 7, 9, 10], [3, 6, 9, 10], [3, 7, 9, 10], [4, 6, 9, 10], [4, 8, 9, 10]]


In [72]:
#Sun graph switching
k = 1
n = 10
m = 2
Q_half = R_sg(n,m)
Q_half = Q_half.T
# MILP_search(Q_half,k,n+m)
# exhaustive_search(Q_half,k,n+m)

# Sun graph switching small test
ed_1 = [ [1, 0, 11], [4, 0, 11], [6, 0, 11], [7, 0, 11], [9, 0, 11] ]  
A_1 = hypergraph_to_tensor(ed_1, n+m)
A_2 = tensor_congruence_transform(A_1,Q_half)
ed_2 = tensor_to_hypergraph_edges(A_2)
print(ed_2)

[[0, 2, 11], [0, 4, 11], [0, 5, 11], [0, 7, 11], [0, 9, 11]]


In [73]:
#GM-switching
k = 2

Q_half = R_gm(4)
Q_half = Q_half.T
n = len(Q_half[0]) 

# MILP_search(Q_half,k,n)
# exhaustive_search(Q_half,k,n)
# 
# GM switching small test
I_1 = identity_matrix(QQ, 1)
Q_half = block_diagonal_matrix(I_1, Q_half)

I_2 = identity_matrix(QQ, 2)
Q_half = block_diagonal_matrix(Q_half, I_2)

m = 3
ed_1 = [ [0, 5, 2], [0, 5, 3], [0, 6, 2], [0, 6, 4], [5, 6, 3], [5, 6, 4] ]
A_1 = hypergraph_to_tensor(ed_1, n+m)
A_2 = tensor_congruence_transform(A_1,Q_half)
ed_2 = tensor_to_hypergraph_edges(A_2)
print(ed_2)


[[0, 1, 5], [0, 1, 6], [0, 3, 6], [0, 4, 5], [1, 5, 6], [2, 5, 6]]


In [74]:
#E-WQH switching 
k = 2

Q_half = R_wqh(3)
n = len(Q_half[0])
# MILP_search(Q_half,k,n)
# exhaustive_search(Q_half,k,n)

I_1 = identity_matrix(QQ, 1)
Q_half = block_diagonal_matrix(I_1, Q_half)

m = 1
ed_1 = [ [1,0], [2,0], [3,0] ]

A_1 = hypergraph_to_tensor(ed_1, n+m)
A_2 = tensor_congruence_transform(A_1,Q_half)
ed_2 = tensor_to_hypergraph_edges(A_2)
print(ed_2)

[[0, 4], [0, 5], [0, 6]]
